In [6]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit.shore import ShoreModel, ShoreFit

os.chdir("/home/olivier/Devel/test/samuel/in4michi/")


# Load fractional anisotropy
dti_fa = nib.load("dti_FA.nii.gz")

# Load DTI mask
dti_mask = nib.load("mask.nii.gz")

# Load and adjust tissue segmentation masks
csf_mask = nib.load("fast_pve_0.nii.gz")
gm_mask = nib.load("fast_pve_1.nii.gz")
wm_mask = nib.load("fast_pve_2.nii.gz")

dti_vecs = nib.load("dti_V1.nii.gz")

data = nib.load("complete_data/data.nii")

bvals, bvecs = read_bvals_bvecs("bvals", "bvecs")
gtab = gradient_table(bvals, bvecs)

In [7]:
model = ShoreModel(gtab)
fit = model.fit(data, wm_mask, gm_mask, csf_mask, dti_mask, 
                    dti_fa, dti_vecs, verbose=True)

100%|██████████| 460800/460800 [00:31<00:00, 14409.28it/s] 


In [8]:
fit.old_save('/home/olivier/Devel/test/samuel/in4michi/bonndit_result/')
fit.save("/home/olivier/Devel/test/samuel/in4michi/bonndit_result/"+"fit.pkl")

In [2]:
fit = ShoreFit.load("/home/olivier/Devel/test/samuel/in4michi/bonndit_result/"+"fit.pkl")

In [3]:
out, wmout, gmout, csfout = fit.fodf(data, verbose=True)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/numpy/linalg/linalg.py:1543: RuntimeWarning: divide by zero encountered in true_divide
  return s[..., 0]/s[..., -1]
Optimization:  22%|██▏       | 6/27 [00:00<00:00, 57.81it/s]

[[ 0.         0.         0.       ]
 [ 0.         0.         0.       ]
 [ 0.         0.         0.       ]
 [ 0.4492     0.259316   0.854971 ]
 [ 0.710982  -0.172307  -0.681774 ]
 [-0.377588   0.89793   -0.22616  ]
 [ 0.634051   0.571626   0.52079  ]
 [-0.645306  -0.762471  -0.0470946]
 [ 0.180011   0.186358   0.96585  ]
 [-0.154821  -0.973693   0.167192 ]]
Condition number of M: inf


Optimization: 100%|██████████| 27/27 [00:00<00:00, 62.90it/s]


In [8]:
img = nib.Nifti1Image(out, data.affine)
nib.save(img, '/home/olivier/Devel/test/samuel/in4michi/bonndit_result/odf.nii.gz')

In [14]:
from bonndit.shore import gtab_reorient, gtab_rotate
from dipy.core.gradients import GradientTable
import numpy as np
GRADIENTS = np.array([[2500,0,0],
                      [0,2500,0],
                      [0,0,2500],
                      [0,0,0]])
gtab = GradientTable(GRADIENTS)
old_vec = np.array((0, 0, 1))
new_vec = np.array((1, 0, 0))
gtab_reorient(gtab, old_vec, new_vec).gradients

array([[    0.,     0., -2500.],
       [    0.,  2500.,     0.],
       [ 2500.,     0.,     0.],
       [    0.,     0.,     0.]])